In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


In [3]:
import numpy as np
import torch

def projectile_trajectory_2d(m, dt, theta, v0, g = 9.81, x0 = 0, y0 = 0, N=1, dim=2):
    """
    Calculates a 2D cartesian trajectory of a projectile of mass m launched at angle theta with initial velocity v0.  
    
        Parameters:
    theta (int or float): angle between projectile trajectory and x-axis at t=0s in degrees \\
    v0 (int or float): the intial velocity in m/s \\
    m (int or float): mass of the projectile in kg \\
    dt (int or float): time step for simulation in s \\
    g (int or float): gravity constant (standard is 9.81) in m/s^2 \\
    x0, y0 (int or float): initial position of ball in m \\

        Returns: 
    The output is a numpy array containing the (x,y,t) coordinates of the projectile
    """

    # Convert theta to radians
    theta_rad = np.radians(theta)

    # Initial velocity components
    vx0 = v0 * np.cos(theta_rad)
    vy0 = v0 * np.sin(theta_rad)

    # Time array
    t_max = (vy0 + np.sqrt(vy0**2+2*g*y0)) / g
    t = np.arange(0, t_max, dt)
    T = len(t)

    # Compute trajectory
    x = x0 + vx0 * t
    y = y0 + vy0 * t - 0.5 * g * t**2

    # Remove negative y values and make np array
    valid = y >= 0
    x, y, t = x[valid], y[valid], t[valid]
    trajectory = np.column_stack((x,y))

    vx = np.full_like(x, vx0)
    vy = vy0 - g * t
    trajectory_velocities = np.column_stack((vx,vy))
    
    trajectory_data = {
        "time": torch.arange(T, dtype=torch.float32),
        "positions": torch.zeros((T, N, dim), dtype=torch.float32),
        "velocities": torch.zeros((T, N, dim), dtype=torch.float32),
        "masses": torch.arange(N, dtype=torch.float32)
    }
    
    trajectory_data["time"] = torch.tensor(t)
    trajectory_data["positions"] = torch.tensor(trajectory[:, :].reshape(T, N, dim))
    trajectory_data["velocities"] = torch.tensor(trajectory_velocities[:, :].reshape(T, N, dim))
    trajectory_data["masses"] = torch.tensor(m)

    return trajectory_data

In [4]:
traj_freefall = projectile_trajectory_2d(3, 0.2, 30, 50)

In [5]:
print(traj_freefall["masses"].numel())

1


In [30]:
def node_data_list(trajectory_dict, self_loop=True, complete_graph=True):
    data_list = []

    N = trajectory_dict["masses"].numel()
    # Ensure masses has the correct shape (N, 1)
    mass_data = trajectory_dict["masses"]

    # If masses is scalar (0-dimensional), repeat it for N particles
    if mass_data.dim() == 0:  # scalar case
        mass_data = mass_data.repeat(N, 1)  # shape (N, 1)

    # If masses is already a 1D tensor of shape (N,)
    elif mass_data.dim() == 1:  # 1D tensor case
        mass_data = mass_data.unsqueeze(1)  # shape (N, 1)
    
    for i in range(len(trajectory_dict["time"]) - 1):
        time_feature = trajectory_dict["time"][i].repeat(N,1)
        position_features = trajectory_dict["positions"][i]

        x_features = torch.cat([
            trajectory_dict["positions"][i],    # (N,dim)
            trajectory_dict["velocities"][i],   # (N, dim)
            mass_data,       # (N, 1)
            time_feature    # (N, 1)
        ], dim=1)   # tensor of shape (N, 2 + 2 + 1 + 1) + (N, 6) if dim = 2

        velocity_update = trajectory_dict["velocities"][i+1] - trajectory_dict["velocities"][i]
        
        y_target = torch.tensor(velocity_update, dtype=torch.float)

        # edge_index = torch.empty((0, 2), dtype=torch.int64)
        
        edge_list = []
        
        if self_loop:
            edge_list.extend([[j,j] for j in range(N)])
            # edge_index = torch.arange(N).unsqueeze(1).repeat(1, 2)
        
        if complete_graph:
            edge_list.extend([k,j] for k in range(N) for j in range(N) if k != j)
            # for j in range(N):
            #     for k in range(N):
            #         if j != k:
            #             edge_list.append([j,k])
            
            
        edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()    

        data_list.append(Data(x=x_features, y=y_target, edge_index=edge_index))

    return data_list

In [31]:
node_data_list(traj_freefall, self_loop=True, complete_graph=False)

/var/folders/_p/ztk2c0b93nbb9q83wg0_pq780000gn/T/ipykernel_48858/2296259503.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_target = torch.tensor(velocity_update, dtype=torch.float)


[Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1], y=[1, 2]),
 Data(x=[1, 6], edge_index=[2, 1],

In [17]:
edge_list = []   
edge_list.extend([i,j] for i in range(3) for j in range(3) if i != j)
# for j in range(3):
#     for k in range(3):
#         if j != k:
#             edge_list.append([j,k])

edge_index = torch.tensor(edge_list, dtype=torch.int64).contiguous()
                

In [18]:
edge_index

tensor([[0, 1],
        [0, 2],
        [1, 0],
        [1, 2],
        [2, 0],
        [2, 1]])

In [ ]:
m = np.array([1,2,3])
dict_ex = {
        "masses": torch.arange(3, dtype=torch.float32)
    }
    
dict_ex["masses"] = torch.tensor(m)
mass_data = dict_ex["masses"]

mass_data = mass_data.unsqueeze(1)  # shape (N, 1)
mass_data

In [40]:
mass_data

tensor([[1],
        [2],
        [3]])

In [36]:
print(numpy.__file__)

NameError: name 'numpy' is not defined